# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
path = "../output_data/cities.csv"
cities_df = pd.read_csv(path)
cities_df = cities_df[['City','Country','Date', 'Lat', 'Lng', 'Max Temp (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)']]

cities_df.head()


,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Lebu,CL,1599265539,-37.62,-73.65,47.07,73,88,4.36
1,New Norfolk,AU,1599265539,-42.78,147.06,55.99,55,41,3.00
2,Bengkulu,ID,1599265540,-3.80,102.27,75.42,89,95,5.30
3,Broome,US,1599265281,42.25,-75.83,66.00,55,1,8.05
4,Cape Town,ZA,1599265318,-33.93,18.42,57.00,93,75,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_df[["Lat", "Lng"]]
weights = cities_df["Humidity (%)"]

In [37]:
figure_layout = {
    'width': "100%",
    'height': '400px',
    'padding': '1px',
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity = max(weights),point_radius = 3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

#DATAFRAME with the given conditions
cities2_df = cities_df.loc[(cities_df["Max Temp (F)"] >= 70) & (cities_df["Max Temp (F)"] <= 80) &
                            (cities_df["Wind Speed (mph)"] <= 10) & (cities_df["Cloudiness (%)"] == 0)].dropna()

cities2_df

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
10,Arraial do Cabo,BR,1599265542,-22.97,-42.03,77.00,69,0,8.05
24,Marsá Maţrūḩ,EG,1599265547,31.35,27.25,78.80,83,0,6.93
38,Mitsamiouli,KM,1599265551,-11.38,43.28,74.95,74,0,8.30
61,Fiumicino,IT,1599265562,41.77,12.23,71.01,100,0,4.70
102,Tabas,IR,1599265578,33.60,56.92,73.40,17,0,2.24
173,Avarua,CK,1599265605,-21.21,-159.78,78.80,83,0,9.17
214,Bat Khela,PK,1599265620,34.62,71.97,73.20,69,0,3.22
246,Safi,MA,1599265632,32.30,-9.24,70.16,85,0,0.18
332,Genoa,IT,1599265664,44.41,8.96,72.00,68,0,2.24
352,Murgab,TM,1599265673,37.50,61.97,71.60,37,0,3.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
#Store into variable named hotel_df
hotel_df = cities2_df[['City','Country','Lat', 'Lng']].copy()
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
10,Arraial do Cabo,BR,-22.97,-42.03,
24,Marsá Maţrūḩ,EG,31.35,27.25,
38,Mitsamiouli,KM,-11.38,43.28,
61,Fiumicino,IT,41.77,12.23,
102,Tabas,IR,33.60,56.92,
173,Avarua,CK,-21.21,-159.78,
214,Bat Khela,PK,34.62,71.97,
246,Safi,MA,32.30,-9.24,
332,Genoa,IT,44.41,8.96,
352,Murgab,TM,37.50,61.97,


In [42]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

#for loop through eaxch city
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    #Requests
    hotel_data = requests.get(base_url, params).json()
    #hotel_data 
    
    #Exception handling to save the first hotel name
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"{row['City']} City and the nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")

Arraial do Cabo City and the nearest hotel is Orlanova Hotel
-------------------------------------------------------------------------
Marsá Maţrūḩ City and the nearest hotel is فندق ديلمار مطروح - Delmar Matrouh Hotel
-------------------------------------------------------------------------
Mitsamiouli City and the nearest hotel is Baobab Magic - Backpacker & Resto
-------------------------------------------------------------------------
Fiumicino City and the nearest hotel is Guest House La Lira
-------------------------------------------------------------------------
Tabas City and the nearest hotel is Amir hotel
-------------------------------------------------------------------------
Avarua City and the nearest hotel is Pacific Resort Rarotonga
-------------------------------------------------------------------------
Bat Khela City and the nearest hotel is Trai Hotel
-------------------------------------------------------------------------
Safi City and the nearest hotel is RIAD L

In [30]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
10,Arraial do Cabo,BR,-22.97,-42.03,Orlanova Hotel
24,Marsá Maţrūḩ,EG,31.35,27.25,فندق ديلمار مطروح - Delmar Matrouh Hotel
38,Mitsamiouli,KM,-11.38,43.28,Baobab Magic - Backpacker & Resto
61,Fiumicino,IT,41.77,12.23,Guest House La Lira
102,Tabas,IR,33.60,56.92,Amir hotel
173,Avarua,CK,-21.21,-159.78,The Rarotongan Beach Resort & Lagoonarium
214,Bat Khela,PK,34.62,71.97,Trai Hotel
246,Safi,MA,32.30,-9.24,RIAD LE CHEVAL BLANC - HÔTEL
332,Genoa,IT,44.41,8.96,Grand Hotel Savoia
352,Murgab,TM,37.50,61.97,


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))